# GLP-1 Safety Surveillance: Descriptive Analyses

This notebook reads the curated CSVs (Reports.csv, Drugs.csv, Reactions.csv) and generates:
- Report volumes over time
- Top reaction terms (overall and by drug role)
- Outcome severity distributions
- Drug role breakdown

**Note:** FAERS is a voluntary reporting system. These analyses cannot establish causation or calculate incidence rates.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Load data - adjust path if running from deliverables/ vs notebooks/
try:
    deliverables_dir = Path('.')
    reports = pd.read_csv(deliverables_dir / 'Reports.csv')
except FileNotFoundError:
    deliverables_dir = Path('../deliverables')
    reports = pd.read_csv(deliverables_dir / 'Reports.csv')

drugs = pd.read_csv(deliverables_dir / 'Drugs.csv')
reactions = pd.read_csv(deliverables_dir / 'Reactions.csv')

# Parse dates
reports['received_date'] = pd.to_datetime(reports['received_date'], errors='coerce')
reports['year'] = reports['received_date'].dt.year
reports['month'] = reports['received_date'].dt.to_period('M').astype(str)
reports['quarter'] = reports['received_date'].dt.to_period('Q').astype(str)

print(f"Loaded {len(reports):,} reports, {len(drugs):,} drug entries, {len(reactions):,} reactions")
reports.head()

## 1. Report Volumes Over Time

In [ ]:
# Monthly report counts
monthly_counts = reports.groupby('month').size()

plt.figure(figsize=(12, 5))
monthly_counts.plot(kind='line', marker='o', markersize=3)
plt.title('GLP-1 Adverse Event Reports Over Time (2024)')
plt.xlabel('Month')
plt.ylabel('Number of Reports')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nQuarterly breakdown:")
reports.groupby('quarter').size()

## 2. Top Reaction Terms

In [ ]:
# Top 20 reactions overall
top_reactions = reactions['reaction_term_text'].value_counts().head(20)

plt.figure(figsize=(10, 8))
top_reactions.plot(kind='barh')
plt.title('Top 20 Reported Adverse Reactions (GLP-1 Agonists)')
plt.xlabel('Number of Reports')
plt.ylabel('Reaction Term')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

top_reactions

## 3. Drug Role Analysis

In [ ]:
# Drug role breakdown
role_counts = drugs['drug_role'].value_counts()

plt.figure(figsize=(8, 5))
role_counts.plot(kind='bar', color=['#e74c3c', '#3498db', '#95a5a6'])
plt.title('Drug Role Distribution')
plt.xlabel('Drug Role')
plt.ylabel('Count')
plt.xticks(rotation=0)
for i, v in enumerate(role_counts.values):
    plt.text(i, v + 500, f'{v:,}', ha='center')
plt.tight_layout()
plt.show()

print("PRIMARY = Suspect drug (GLP-1 suspected of causing reaction)")
print("SECONDARY = Concomitant drug (taken alongside)")
print("ASSOCIATED = Other associated drug")
print(f"\nTotal drug entries: {len(drugs):,}")
role_counts

In [ ]:
# Top reactions for PRIMARY (suspect) drugs only
primary_reports = drugs[drugs['drug_role'] == 'PRIMARY']['safetyreportid'].unique()
primary_reactions = reactions[reactions['safetyreportid'].isin(primary_reports)]

top_primary = primary_reactions['reaction_term_text'].value_counts().head(15)

plt.figure(figsize=(10, 6))
top_primary.plot(kind='barh', color='#e74c3c')
plt.title('Top Reactions Where GLP-1 is PRIMARY (Suspect) Drug')
plt.xlabel('Number of Reports')
plt.ylabel('Reaction Term')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

top_primary

## 4. Outcome Severity Distribution

In [ ]:
# Outcome severity counts
outcomes = ['death', 'hospitalization', 'life_threatening', 'disability', 'congenital_anomaly']
outcome_counts = reports[outcomes].sum().sort_values(ascending=True)

plt.figure(figsize=(10, 5))
outcome_counts.plot(kind='barh', color='#9b59b6')
plt.title('Serious Outcome Distribution')
plt.xlabel('Number of Reports')
plt.ylabel('Outcome Type')
plt.tight_layout()
plt.show()

print("\nOutcome counts:")
outcome_counts.sort_values(ascending=False)

## 5. Demographics Summary

In [ ]:
# Sex distribution
sex_counts = reports['patient_sex'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Sex pie chart
axes[0].pie(sex_counts.values, labels=sex_counts.index, autopct='%1.1f%%', colors=['#3498db', '#e74c3c', '#95a5a6'])
axes[0].set_title('Patient Sex Distribution')

# Age histogram
reports['patient_age_years'].dropna().hist(bins=30, ax=axes[1], color='#2ecc71', edgecolor='black')
axes[1].set_title('Patient Age Distribution')
axes[1].set_xlabel('Age (years)')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()

print(f"Sex distribution: {dict(sex_counts)}")
print(f"Age - Mean: {reports['patient_age_years'].mean():.1f}, Median: {reports['patient_age_years'].median():.1f}")
print(f"Age - Missing: {reports['patient_age_years'].isna().sum():,} ({reports['patient_age_years'].isna().mean()*100:.1f}%)")

## 6. Summary Statistics

In [ ]:
print("=" * 50)
print("GLP-1 SAFETY SURVEILLANCE DATASET SUMMARY")
print("=" * 50)
print(f"\nTotal Reports: {len(reports):,}")
print(f"Total Drug Entries: {len(drugs):,}")
print(f"Total Reaction Entries: {len(reactions):,}")
print(f"\nDate Range: {reports['received_date'].min().date()} to {reports['received_date'].max().date()}")
print(f"\nUnique Reaction Terms: {reactions['reaction_term_text'].nunique():,}")
print(f"Unique Drug Names: {drugs['drug_name_original'].nunique():,}")
print("\n" + "=" * 50)
print("LIMITATIONS")
print("=" * 50)
print("- FAERS is voluntary; cannot calculate incidence rates")
print("- Reports do not establish causation")
print("- Subject to reporting bias and duplicate submissions")